
<br>
<font>
<div dir=ltr align=center>
<img src="https://cdn.freebiesupply.com/logos/large/2x/sharif-logo-png-transparent.png" width=150 height=150> <br>
<font color=0F5298 size=7>
Artificial Intelligence <br>
<font color=2565AE size=5>
Computer Engineering Department <br>
Spring 2024<br>
<font color=3C99D size=5>
Practical Assignment - Minimax <br>
<font color=696880 size=4>
Ali Aghayari


____

# P0 : Game explanation and environment setup (0 points)

In this Jupyter notebook, we aim to develop the AI logic for the game **Connect4**. Players take turns dropping pieces into a grid, and the first player to align four consecutive pieces vertically, horizontally, or diagonally wins. The focus is on creating an intelligent AI opponent using the minimax algorithm with alpha-beta pruning to deliver a challenging gameplay experience.

<span style="color: red;">However, there’s an added twist: after either player drops a piece, there is a 12.5% chance that the entire board will rotate 90 degrees clockwise, changing the direction of gravity as well! Keep this rule in mind when implementing your heuristic, as your AI will face a tough challenge ahead.</span>

<br>

Note: The winning condition will be checked after applying the rotation. If both sides have winning conditions, the player who made the last move will lose.

<br>
Note: For full clarification, the following occurs:
- Some players make their moves.
- Rotations happens. (12.5%)
- New gravity is applied to every piece on the board. (12.5%)
- The winning condition is then checked.

<br>
Rules: Do not modify the code cells that don’t have TODO comments, except for those that are explicitly mentioned as okay to change.

First, we will define some constants to make the code cleaner, more organized, and to set up the game environment.

In [1]:
# Game Constants
ROW_COUNT = 7
COLUMN_COUNT = 7
WINDOW_LENGTH = 4
EMPTY = 0

# Players and Pieces
PLAYER = 0
AI = 1
EMPTY_PIECE = 0
PLAYER_PIECE = 1
AI_PIECE = 2

# Colors (RGB values) - you can change the colors to your liking
FG_COLOR = (0, 0, 255)
BG_COLOR = (0, 0, 0)
P1_COLOR = (255, 0, 0)
P2_COLOR = (0, 255, 0)

# Pygame Constants
SQUARESIZE = 80
RADIUS = SQUARESIZE // 2 - 5
SCREEN_WIDTH = COLUMN_COUNT * SQUARESIZE
SCREEN_HEIGHT = (ROW_COUNT + 1) * SQUARESIZE

These libraries are sufficient to complete this task, but feel free to add any additional imports you may need.

In [4]:
!pip install pygame

import numpy as np
import random
import pygame
import math
import time

# Additional libraries can be imported here if needed


pygame 2.6.1 (SDL 2.28.4, Python 3.12.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


# P1 : Util functions (20 points)

We need to initialize the game board as an empty 2D array with dimensions of ROW_COUNT by COLUMN_COUNT.

In [8]:
def create_board():
    # TODO: Follow the instructions as described.
    board = np.zeros((ROW_COUNT, COLUMN_COUNT), dtype=int)
    return board

Fill in the code to find the valid columns where a piece can be dropped.

In [11]:
def is_valid_location(board, col):
    # TODO: Follow the instructions as described.
    is_valid = board[ROW_COUNT-1][col] == EMPTY
    return is_valid

In [13]:
def get_valid_locations(board):
    # TODO: Follow the instructions as described.
    return [col for col in range(COLUMN_COUNT) if is_valid_location(board, col)]

Fill in the code to find the valid row for the given column where a piece can be dropped.

In [16]:
def get_next_open_row(board, col):
    # TODO: Follow the instructions as described.
    for row in range(0, ROW_COUNT, 1):
        if board[row][col] == EMPTY:
            return row
    return None

Fill in the code to drop a piece in the specified column of the board.

In [19]:
def drop_piece(board, col, piece):
    # TODO: Follow the instructions as described.
    row = get_next_open_row(board, col)
    if row is not None: 
        board[row][col] = piece

Fill in the code to check if the specified piece has won the game.

In [22]:
def winning_move(board, piece):
    # TODO: Follow the instructions as described.

    for row in range(ROW_COUNT):
        for col in range(COLUMN_COUNT - 3):
            if all(board[row][col + i] == piece for i in range(WINDOW_LENGTH)):
                return True

    for col in range(COLUMN_COUNT):
        for row in range(ROW_COUNT - 3):
            if all(board[row + i][col] == piece for i in range(WINDOW_LENGTH)):
                return True

    for row in range(ROW_COUNT - 3):
        for col in range(COLUMN_COUNT - 3):
            if all(board[row + i][col + i] == piece for i in range(WINDOW_LENGTH)):
                return True

    for row in range(3, ROW_COUNT):
        for col in range(COLUMN_COUNT - 3):
            if all(board[row - i][col + i] == piece for i in range(WINDOW_LENGTH)):
                return True

    return False

# P2 : Scoring function and Minimax implementation (50 points)

Fill in the code to score the current situation for the player. Hint: You can divide the board into separate windows, score each window individually for the given piece, and sum the scores to obtain the total board score for that piece.

In [53]:
def evaluate_window(window, piece):
    score = 0
    opp_piece = PLAYER_PIECE if piece == AI_PIECE else AI_PIECE

    if window.count(piece) == 4:
        score += 100  
    elif window.count(piece) == 3 and window.count(EMPTY) == 1:
        score += 50  
    elif window.count(piece) == 2 and window.count(EMPTY) == 2:
        score += 10  

    if window.count(opp_piece) == 3 and window.count(EMPTY) == 1:
        score -= 30

    return score


In [28]:
def score_position(board, piece):
    # TODO: Follow the instructions as described.
    # NOTE: Split the board into separate windows and sum up the scores for each window.
    # NOTE: Adjust the scoring logic to account for the additional game rules.
    # return aggregated score
    score = 0
    
    center_array = [int(board[row][COLUMN_COUNT // 2]) for row in range(ROW_COUNT)]
    center_count = center_array.count(piece)
    score += center_count * 6  

    for row in range(ROW_COUNT):
        for col in range(COLUMN_COUNT - 3):
            window = [board[row][col+i] for i in range(4)]
            score += evaluate_window(window, piece)

    for row in range(ROW_COUNT - 3):
        for col in range(COLUMN_COUNT):
            window = [board[row+i][col] for i in range(4)]
            score += evaluate_window(window, piece)

    for row in range(ROW_COUNT - 3):
        for col in range(COLUMN_COUNT - 3):
            window = [board[row+i][col+i] for i in range(4)]
            score += evaluate_window(window, piece)

    for row in range(ROW_COUNT - 3):
        for col in range(3, COLUMN_COUNT):
            window = [board[row+i][col-i] for i in range(4)]
            score += evaluate_window(window, piece)

    return score

Fill in the code to implement the minimax algorithm with alpha beta pruning using the utility function provided.

In [31]:
def minimax(board, depth, alpha, beta, maximizingPlayer):
    # TODO: Retrieve the list of valid moves
    # TODO: Return the appropriate value if a terminal condition (win/loss/draw/max_depth) is met
    # TODO: Check whose turn it is, and apply minimax logic for that player; recursively call with decreased depth for the opponent
    # TODO: Implement alpha-beta pruning to optimize the search
    # NOTE: Be careful not to alter the original board during recursive exploration
    # NOTE: This function is supposed to find the best column to drop a piece on
    # Return best column to drop and the value associated to this move
    # Return best_column, value
    valid_locations = get_valid_locations(board)
    player_won = winning_move(board, PLAYER_PIECE)
    AI_won = winning_move(board, AI_PIECE)
    draw = len(valid_locations) == 0
    is_terminal_condition = player_won or AI_won or draw

    if depth == 0 or is_terminal_condition:
        if is_terminal_condition:
            if AI_won:
                return (None, 100000000000000)
            elif player_won:
                return (None, -100000000000000)
            else:
                return (None, 0)
        else:
            return (None, score_position(board, AI_PIECE))

    if maximizingPlayer:
        value = -math.inf
        best_column = random.choice(valid_locations)
        for col in valid_locations:
            row = get_next_open_row(board, col)
            board_copy = board.copy()
            drop_piece(board_copy, col, AI_PIECE)
            new_score = minimax(board_copy, depth - 1, alpha, beta, False)[1]
            if new_score > value:
                value = new_score
                best_column = col
            alpha = max(alpha, value)
            if alpha >= beta:
                break
        return best_column, value

    else:
        value = math.inf
        best_column = random.choice(valid_locations)
        for col in valid_locations:
            row = get_next_open_row(board, col)
            board_copy = board.copy()
            drop_piece(board_copy, col, PLAYER_PIECE)
            new_score = minimax(board_copy, depth - 1, alpha, beta, True)[1]
            if new_score < value:
                value = new_score
                best_column = col
            beta = min(beta, value)
            if alpha >= beta:
                break
        return best_column, value

# P3 : PVE (0 points)

Run this code to test your AI's performance. It is recommended to execute this part locally.

In [35]:
pygame.init()
width = COLUMN_COUNT * SQUARESIZE
height = (ROW_COUNT + 1) * SQUARESIZE
size = (width, height)
screen = pygame.display.set_mode(size)
myfont = pygame.font.SysFont("monospace", 75)

def rotate_board(board):
    rotated_board = np.rot90(board)
    for col in range(rotated_board.shape[1]):
        column = rotated_board[:, col]
        non_empty = column[column != 0]
        empty_count = ROW_COUNT - len(non_empty)
        rotated_board[:, col] = np.concatenate((non_empty, np.zeros(empty_count)))
    return rotated_board

def draw_board(board):
    for c in range(COLUMN_COUNT):
        for r in range(ROW_COUNT):
            pygame.draw.rect(screen, FG_COLOR, (c * SQUARESIZE, r * SQUARESIZE + SQUARESIZE, SQUARESIZE, SQUARESIZE))
            pygame.draw.circle(screen, BG_COLOR, (
                int(c * SQUARESIZE + SQUARESIZE / 2), int(r * SQUARESIZE + SQUARESIZE + SQUARESIZE / 2)), RADIUS)

    for c in range(COLUMN_COUNT):
        for r in range(ROW_COUNT):
            if board[r][c] == PLAYER_PIECE:
                pygame.draw.circle(screen, P1_COLOR, (
                    int(c * SQUARESIZE + SQUARESIZE / 2), height - int(r * SQUARESIZE + SQUARESIZE / 2)), RADIUS)
            elif board[r][c] == AI_PIECE:
                pygame.draw.circle(screen, P2_COLOR, (
                    int(c * SQUARESIZE + SQUARESIZE / 2), height - int(r * SQUARESIZE + SQUARESIZE / 2)), RADIUS)
    pygame.display.update()

def run_game():
    board = create_board()
    draw_board(board)
    game_over = False
    turn = random.choice([0, 1])
    while not game_over:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                return

            if event.type == pygame.MOUSEMOTION:
                pygame.draw.rect(screen, BG_COLOR, (0, 0, width, SQUARESIZE))
                posx = event.pos[0]
                if turn == PLAYER:
                    pygame.draw.circle(screen, P1_COLOR, (posx, int(SQUARESIZE / 2)), RADIUS)

            pygame.display.update()

            if event.type == pygame.MOUSEBUTTONDOWN:
                pygame.draw.rect(screen, BG_COLOR, (0, 0, width, SQUARESIZE))
                if turn == PLAYER:
                    posx = event.pos[0]
                    col = int(math.floor(posx / SQUARESIZE))

                    if is_valid_location(board, col):
                        drop_piece(board, col, PLAYER_PIECE)
                        if random.random() <= 0.125:
                            board = rotate_board(board)
                            draw_board(board)
                            pygame.time.wait(1500)

                        if winning_move(board, AI_PIECE):
                            label = myfont.render("Player 2 wins!!", 1, P2_COLOR)
                            print("Player 2 wins!!")
                            screen.blit(label, (40, 10))
                            game_over = True
                        elif winning_move(board, PLAYER_PIECE):
                            label = myfont.render("Player 1 wins!!", 1, P1_COLOR)
                            print("Player 1 wins!!")
                            screen.blit(label, (40, 10))
                            game_over = True

                        turn += 1
                        turn = turn % 2
                        draw_board(board)

        if turn == AI and not game_over:
            col, minimax_score = minimax(board, 5, -math.inf, math.inf, True)

            if is_valid_location(board, col):
                drop_piece(board, col, AI_PIECE)

                if random.random() <= 0.125:
                    board = rotate_board(board)
                    draw_board(board)
                    pygame.time.wait(1500)

                if winning_move(board, PLAYER_PIECE):
                    label = myfont.render("Player 1 wins!!", 1, P1_COLOR)
                    print("Player 1 wins!!")
                    screen.blit(label, (40, 10))
                    game_over = True
                elif winning_move(board, AI_PIECE):
                    label = myfont.render("Player 2 wins!!", 1, P2_COLOR)
                    print("Player 2 wins!!")
                    screen.blit(label, (40, 10))
                    game_over = True

                turn += 1
                turn = turn % 2
                draw_board(board)

        if game_over:
            pygame.time.wait(3000)
            pygame.quit()
            return


run_game()

Player 2 wins!!


# P4 : EVE (30 points)

In this section, we will simulate an AI battle where your AI heuristic should outperform our provided heuristic. Don’t worry; the opposing AI is not optimal, but if your scoring approach is inadequate, you may lose some credit from P2. Your AI should demonstrate a significant advantage, meaning it should consistently beat our AI on average, regardless of whether it plays as the first or second player. Please note that your search tree should can't have a higher depth than our heuristic.

Implement the minimax algorithm similar to your main minimax function. However, do not modify the tester_evaluate_window and tester_score_position functions. Remember to utilize tester_score_position within tester_minimax!

In [37]:
def tester_evaluate_window(window, piece):
    score = 0
    opp_piece = PLAYER_PIECE if piece == AI_PIECE else AI_PIECE
    if window.count(piece) == 4:
        score += 100
    elif window.count(piece) == 3 and window.count(EMPTY) == 1:
        score += 5
    elif window.count(piece) == 2 and window.count(EMPTY) == 2:
        score += 2
    if window.count(opp_piece) == 3 and window.count(EMPTY) == 1:
        score -= 4
    return score


def tester_score_position(board, piece):
    score = 0
    center_array = [int(i) for i in list(board[:, COLUMN_COUNT // 2])]
    score += center_array.count(piece) * 3

    for r in range(ROW_COUNT):
        row_array = [int(i) for i in list(board[r, :])]
        for c in range(COLUMN_COUNT - 3):
            window = row_array[c:c + WINDOW_LENGTH]
            score += tester_evaluate_window(window, piece)

    for c in range(COLUMN_COUNT):
        col_array = [int(i) for i in list(board[:, c])]
        for r in range(ROW_COUNT - 3):
            window = col_array[r:r + WINDOW_LENGTH]
            score += tester_evaluate_window(window, piece)

    for r in range(ROW_COUNT - 3):
        for c in range(COLUMN_COUNT - 3):
            window = [board[r + i][c + i] for i in range(WINDOW_LENGTH)]
            score += tester_evaluate_window(window, piece)

    for r in range(ROW_COUNT - 3):
        for c in range(COLUMN_COUNT - 3):
            window = [board[r + 3 - i][c + i] for i in range(WINDOW_LENGTH)]
            score += tester_evaluate_window(window, piece)


def tester_minimax(board, depth, alpha, beta, maximizingPlayer):
    # TODO: Implement using your main minimax logic
    # NOTE: Remember to replace score_position with tester_score_position!!!
    # NOTE: In this function, you represent the "AI" and the tester represents the "PLAYER".
    #       No further changes are needed if you haven't modified the constants.

    valid_locations = get_valid_locations(board)
    player_won = winning_move(board, PLAYER_PIECE)
    AI_won = winning_move(board, AI_PIECE)
    draw = len(valid_locations) == 0

    is_terminal_condition = player_won or AI_won or draw

    if depth == 0 or is_terminal_condition:
        if AI_won:
            return None, 1000000 
        elif player_won:
            return None, -1000000  
        else:
            return None, 0 

    if maximizingPlayer:
        max_eval = -math.inf
        best_col = random.choice(valid_locations)
        for col in valid_locations:
            row = get_next_open_row(board, col)
            temp_board = board.copy()
            drop_piece(temp_board, col, AI_PIECE)
            _, score = tester_minimax(temp_board, depth - 1, alpha, beta, False)
            if score > max_eval:
                max_eval = score
                best_col = col
            alpha = max(alpha, score)
            if alpha >= beta:
                break
        return best_col, max_eval

    else:
        min_eval = math.inf
        best_col = random.choice(valid_locations)
        for col in valid_locations:
            row = get_next_open_row(board, col)
            temp_board = board.copy()
            drop_piece(temp_board, col, PLAYER_PIECE)
            _, score = tester_minimax(temp_board, depth - 1, alpha, beta, True)
            if score < min_eval:
                min_eval = score
                best_col = col
            beta = min(beta, score)
            if alpha >= beta:
                break
        return best_col, min_eval


In [39]:
def simulate_game():
    board = create_board()
    starting_turn = turn = random.choice([0, 1])

    game_over = False
    while not game_over:
        if turn == PLAYER:
            col, minimax_score = tester_minimax(board, 4, -math.inf, math.inf, True)
            if is_valid_location(board, col):
                drop_piece(board, col, PLAYER_PIECE)
                if random.random() <= 0.125:
                    board = rotate_board(board)

                if winning_move(board, AI_PIECE):
                    return 1, starting_turn

                elif winning_move(board, PLAYER_PIECE):
                    return 0, starting_turn
                turn += 1
                turn = turn % 2

        if turn == AI and not game_over:
            col, minimax_score = minimax(board, 4, -math.inf, math.inf, True)
            if is_valid_location(board, col):
                drop_piece(board, col, AI_PIECE)
                if random.random() <= 0.125:
                    board = rotate_board(board)
                if winning_move(board, PLAYER_PIECE):
                    return 0, starting_turn
                elif winning_move(board, AI_PIECE):
                    return 1, starting_turn

                turn += 1
                turn = turn % 2

Run this tester. You need to win at least 65% of the games to pass. 
<br>
The code execution should take less than 10 minutes to complete. If it exceeds this time, performance optimization might be necessary.

In [55]:
wins = 0
tests = 100
throw = 0
disadvantage = 0

start_time = time.time()

for i in range(tests):
    result = simulate_game()
    if result[1] == PLAYER and result[0] == 0: disadvantage += 1
    if result[1] == AI and result[0] == 0: throw += 1
    wins += result[0]

end_time = time.time()

print(f"you won {wins / tests}% of games")
print(f"you throw {throw / tests}% of games")
print(f"you lost logically {disadvantage / tests}% of games")
print(f"Code execution time: {end_time - start_time:.4f} seconds")

you won 0.82% of games
you throw 0.05% of games
you lost logically 0.13% of games
Code execution time: 111.4025 seconds
